In [1]:
from datasets import load_dataset
import datasets
from diffusers import AutoencoderKL
import torch
import torchvision.transforms as transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
datasets.config.HF_HUB_OFFLINE = 1 # Comment this out if you havent downloaded the dataset yet

In [4]:
train_ds = load_dataset("tpremoli/CelebA-attrs", cache_dir="datasets/CelebA-attrs/train", split="train")
validation_ds = load_dataset("tpremoli/CelebA-attrs", cache_dir="datasets/CelebA-attrs/validation", split="validation")
test_ds = load_dataset("tpremoli/CelebA-attrs", cache_dir="datasets/CelebA-attrs/test", split="test")

Using the latest cached version of the dataset since tpremoli/CelebA-attrs couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'default' at datasets\CelebA-attrs\train\tpremoli___celeb_a-attrs\default\0.0.0\ed9021d2871ceddbd3cf0fb642544bd7c60c5152 (last modified on Wed Sep 11 12:26:31 2024).
Using the latest cached version of the dataset since tpremoli/CelebA-attrs couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'default' at datasets\CelebA-attrs\validation\tpremoli___celeb_a-attrs\default\0.0.0\ed9021d2871ceddbd3cf0fb642544bd7c60c5152 (last modified on Mon Aug 26 18:16:43 2024).
Using the latest cached version of the dataset since tpremoli/CelebA-attrs couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'default' at datasets\CelebA-attrs\test\tpremoli___celeb_a-attrs\default\0.0.0\ed9021d2871ced

In [5]:
transform = transforms.Compose([
    transforms.Resize((216, 176)),  # Resize to 176x216 (Height x Width)
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Scale to [-1, 1]
])

In [6]:
def transform_sample(sample):
    sample["image"] = transform(sample["image"])
    return sample

train_ds = train_ds.map(transform_sample)
validation_ds = validation_ds.map(transform_sample)
test_ds = test_ds.map(transform_sample)

In [7]:
train_ds = train_ds.with_format(type='torch', columns=['image'], output_all_columns=True)
validation_ds = validation_ds.with_format(type='torch', columns=['image'], output_all_columns=True)
test_ds = test_ds.with_format(type='torch', columns=['image'], output_all_columns=True)

In [8]:
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", cache_dir="models/vae")
vae = vae.to(device)

In [9]:
transform2 = lambda x: vae.encode(x.to(device)).latent_dist.sample().cpu()

def transform_sample2(sample):
    sample["latents"] = transform2(sample["image"])
    return sample

train_ds = train_ds.map(transform_sample2, batched=True, batch_size=32, remove_columns=["image"])
validation_ds = validation_ds.map(transform_sample2, batched=True, batch_size=32, remove_columns=["image"])
test_ds = test_ds.map(transform_sample2, batched=True, batch_size=32, remove_columns=["image"])

Map:   0%|          | 0/162770 [00:00<?, ? examples/s]

d:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\diffusers\models\attention_processor.py:1584: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  hidden_states = F.scaled_dot_product_attention(


Map:   0%|          | 0/19962 [00:00<?, ? examples/s]

Map:   0%|          | 0/19867 [00:00<?, ? examples/s]

In [10]:
train_ds.save_to_disk("datasets/CelebA-attrs-latents/train")
validation_ds.save_to_disk("datasets/CelebA-attrs-latents/validation")
test_ds.save_to_disk("datasets/CelebA-attrs-latents/test")

Saving the dataset (0/153 shards):   0%|          | 0/162770 [00:00<?, ? examples/s]

Saving the dataset (0/19 shards):   0%|          | 0/19962 [00:00<?, ? examples/s]

Saving the dataset (0/19 shards):   0%|          | 0/19867 [00:00<?, ? examples/s]

In [3]:
# Upload to huggingface
from datasets import load_from_disk

train_ds = load_from_disk("datasets/CelebA-attrs-latents/train")
validation_ds = load_from_disk("datasets/CelebA-attrs-latents/validation")
test_ds = load_from_disk("datasets/CelebA-attrs-latents/test")

Loading dataset from disk:   0%|          | 0/153 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/19 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/19 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/191 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

KeyboardInterrupt: 

In [5]:
train_ds.push_to_hub("SwayStar123/CelebA-attrs-latents", split="train")
validation_ds.push_to_hub("SwayStar123/CelebA-attrs-latents", split="validation")
test_ds.push_to_hub("SwayStar123/CelebA-attrs-latents", split="test")


Uploading the dataset shards:   0%|          | 0/153 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SwayStar123/CelebA-attrs-latents/commit/f5af1fdd421292ed81e80ffb37a85429a02d38ba', commit_message='Upload dataset', commit_description='', oid='f5af1fdd421292ed81e80ffb37a85429a02d38ba', pr_url=None, pr_revision=None, pr_num=None)